# Test DL

**Table of contents**<a id='toc0_'></a>    
- 1. [Imports](#toc1_)    
- 2. [Setup](#toc2_)    
- 3. [Run](#toc3_)    
  - 3.1. [Same](#toc3_1_)    
  - 3.2. [New](#toc3_2_)    
- 4. [MPC](#toc4_)    
- 5. [Euler-errors](#toc5_)    
- 6. [Compute policy on grids](#toc6_)    
- 7. [Timing](#toc7_)    
  - 7.1. [Details](#toc7_1_)    
- 8. [Save/load](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Imports](#toc0_)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext line_profiler

In [3]:
import os
import numpy as np
import torch

In [4]:
import matplotlib.pyplot as plt
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

In [5]:
from EconDLSolvers import choose_gpu

In [6]:
from BufferStockModel import BufferStockModelClass

## 2. <a id='toc2_'></a>[Setup](#toc0_)

In [7]:
algoname = 'DeepFOC'

In [ ]:
device = choose_gpu()

In [9]:
model = BufferStockModelClass(algoname=algoname,device=device,par={'Nstates_fixed':0},train={'K_time':0.5},show_memory=True)

## 3. <a id='toc3_'></a>[Run](#toc0_)

In [ ]:
model.solve(do_print=True,do_print_all=False,show_memory=True)

**Print detailed timings:**

In [ ]:
for k,v in model.info.items():
    if type(k) == str and k[:4] == 'time':
        print(f'{k}: {v:.1f} secs')  

### 3.1. <a id='toc3_1_'></a>[Same](#toc0_)

In [ ]:
model_same = BufferStockModelClass(algoname=algoname,device=device,par={'Nstates_fixed':0},train={'K':model.train.sim_R_freq+1})
model_same.solve(do_print=True)
del model_same

### 3.2. <a id='toc3_2_'></a>[New](#toc0_)

1. New sample for lifetime reward
1. New initial values for neural nets
1. New training samples

In [ ]:
torch_rng_state = torch.get_rng_state()
model_new = BufferStockModelClass(algoname=algoname,device=device,
                                  par={'Nstates_fixed':0},train={'K':model.train.sim_R_freq+1},
                                  torch_rng_state=torch_rng_state)
model_new.solve(do_print=True)
del model_new

## 7. <a id='toc7_'></a>[Timing](#toc0_)

In [ ]:
model_time = BufferStockModelClass(algoname=algoname,device=device,par={'Nstates_fixed':0},train={'K_time':0.5})
model_time.time_solve()

### 7.1. <a id='toc7_1_'></a>[Details](#toc0_)

In [15]:
model_time = BufferStockModelClass(algoname=algoname,device=device,par={'Nstates_fixed':0},train={'K_time':0.5})

In [16]:
if not os.path.isdir('timings'): os.mkdir('timings')

In [ ]:
filename = f'timings/{model_time.train.algoname}.txt'
%lprun -f model_time.solve -T $filename model_time.solve(do_print=False)